In [2]:
import pandas as pd
import numpy as np
# from sklearn.preprocessing import PolynomialFeatures
from sklearn import decomposition
import os
import scipy.stats as sc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC


In [3]:
df_co = pd.read_csv('Transformed Data/Transformed_Co', index_col = 0)
df_pt = pd.read_csv('Transformed Data/Transformed_Pt' , index_col = 0)

In [4]:
df_co_len = df_co.shape[0]
df_pt_len = df_pt.shape[0]

In [5]:
df_co_pca = pd.DataFrame(df_co)
df_pt_pca = pd.DataFrame(df_pt)

In [6]:
y1 = pd.Series([0]*df_co_len)
y1.shape

(92,)

In [7]:
y2 = pd.Series([1]*df_pt_len, index = range(df_co_len-1,(df_co_len + df_pt_len)-1))
y2.shape

(214,)

In [8]:
y = pd.concat([y1,y2]) 
y.shape

(306,)

In [9]:
X = pd.concat([df_co_pca, df_pt_pca])

In [14]:

max_score = 0
best_param = []
for i in range(0,50):
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    clf = SVC(kernel = 'linear', C = 1).fit(X_train, np.ravel(y_train))
    from sklearn.model_selection import GridSearchCV

    grid_values = {'C': [1, 2, 5, 10, 15, 20, 25, 40, 50, 70, 100, 500, 1000]}

    grid_lr = GridSearchCV(clf, param_grid = grid_values, scoring = 'accuracy')
    grid_lr.fit(X_train, y_train)
    if max_score < grid_lr.best_score_:
        max_score = grid_lr.best_score_
        best_param = grid_lr.best_params_

In [15]:
print( max_score)
print( best_param)

0.842794759825
{'C': 1}
